# Lab 2: Landscape Energy Balance

<img src="./assets/mpala1.jpeg" alt="mpala" width="800"/>

In the last lab, we used mass balance concepts to calculate the water balance within a watershed. In this lab, will will use energy balance concepts to explore the fluxes of energy across a landscape (more specifically, $\uparrow$ this landscape!).


<div class="boxhead3">
    Lab 2 Contents
</div>

<div class="boxtext3">
<ul class="a">
    <li> 1. Surface Energy Balance </li>
    <ul class="b">
        <li> Background </li>
    </ul>
    <li> 2. The Radiation Budget </li>
    <ul class="b">
        <li> Shortwave radiation </li>
        <li> Longwave radiation </li>
        <li> Problems </li>
    </ul>
    <li> 3. Landscape Energy Budget </li>
    <ul class="b">
        <li> Ground heat flux </li>
        <li> Sensible heat flux </li>
        <li> Latent heat flux </li>
        <li> Problems </li>
    </ul>
    <li> 4. Putting it all together </li>
</ul>

</div>


<hr style="border-top: 0.2px solid gray; margin-top: 12pt; margin-bottom: 0pt"></hr>

### Instructions
Work through the exercise, writing code where indicated. For some of the questions, you may want to add a Markdown cell to discuss your answer. To run a cell, click on the cell and press "Shift" + "Enter" or click the "Run" button in the toolbar at the top. Note: Do not restart the kernel and clear all outputs. If this happens, run the last cell in the notebook before proceeding.

<p style="color:#408000; font-weight: bold"> 🐍 &nbsp; &nbsp; This symbol designates an important note about Python structure, syntax, or another quirk.  </p>

<p style="color:#443823; font-weight: bold">📊 &nbsp; &nbsp; This symbol designates an important note about environmental data methods, sources, and access. </p>

<p style="color:#008C96; font-weight: bold"> ▶️ &nbsp; &nbsp; This symbol designates a cell with code to be run.  </p>

<p style="color:#008C96; font-weight: bold"> ✏️ &nbsp; &nbsp; This symbol designates a partially coded cell with an example.  </p>

<p style="color:#008C96; font-weight: bold"> 📚 &nbsp; &nbsp; This symbol designates a practice question.  </p>


<hr style="border-top: 1px solid gray; margin-top: 24px; margin-bottom: 1px"></hr>

# 1. Surface Energy Balance

<hr style="border-top: 0.5px solid gray; margin-top: 10pt; margin-bottom: 12pt"></hr>


Much like mass balance, the energy balance of a system consists of the sum of inputs and outputs. While the mass balance terms can be discretized into obvious components (e.g. precipitation), the components of the Earth's energy budget are **fluxes**, meaning that they consist of both inputs *and* outputs and can be either positive or negative, depending on environmental conditions. These fluxes are driven by four important processes: **radiation**, **conduction**, **convection**, and **evaporation**.

The largest and perhaps most obvious component of the Earth's energy budget is the **net radiation**, $R_n$, consisting of electromagnetic radiation that is *absorbed*, *reflected*, and *emitted* by the Earth's surface. The surplus of incoming solar radiation provides excess energy for transport-driven processes at the Earth's surface. Thermal conduction of heat energy within the ground leads to **ground heat flux**, $G$. **Sensible heat flux**, $H$, refers to the turbulent transfer of energy between the surface and the atmosphere via convective processes. **Latent heat flux**, $\lambda E$, represents the loss of energy associated with phase changes between liquid water and water vapor through evaporation, transpiration, and condensation.


\begin{equation}
\tag{1.1}
    \frac{dQ}{dt} \ = \ R_n \ - \ H \ - \ \lambda E \ - \ G
\end{equation}

These energy fluxes (also referred to as *energy flux densities*) quantify the rate of energy transfer per unit area and are measured in units of J m<sup>-2</sup> s<sup>-1</sup> or W m<sup>-2</sup>.

Assuming the system is in *steady state*, a valid assumption over sufficiently long time scales, $ \frac{dQ}{dt} \ = \ 0$, so we can rearrange Equation 1.1 as:

\begin{equation}
\tag{1.2}
    R_n \ = \ H \ + \ \lambda E \ + \ G
\end{equation}

### Measuring environmental fluxes

In this lab, we will be using data collected from a micrometerological tower that uses [eddy covariance](https://en.wikipedia.org/wiki/Eddy_covariance) techniques to measure turbulent fluxes in the atmospheric boundary layer. The physics of this method are too complex to get into here, but eddy covariance is one of the main ways of determining landscape energy balance. These "flux towers" are equipped with sensors to measure wind speed and direction and gas (usually H<sub>2</sub>O and CO<sub>2</sub>) concentrations, and radiation (and usually numerous other sensors!) at very high frequencies (commonly 10 Hz or 10 times per second). These data are then used to calculate fluxes of gas, energy, and momentum across the landscape. 

This short video provides a basic overview of eddy covariance, focusing on CO<sub>2</sub> fluxes. While we won't be working with carbon fluxes in this lab, you can replace "CO<sub>2</sub>" with "H<sub>2</sub>O" to think about water vapor fluxes due to sensible and latent heat.

[<img src="./assets/ec_video.png" alt="ec_video" width="600"/>](https://youtu.be/CR4Anc8Mkas)

### Background

The data we'll work with in this lab come from a flux tower in central Kenya just a few kilometers north of the Equator. Like much of East Africa, the flux tower site is situated in a dryland savanna environment. As we saw in Exercise 3, there are two "rainy" seasons in this part of Kenya, one beginning in March and another in October. 

<img src="./assets/mpalatower.jpeg" alt="tower" width="400"/>

In this lab, we'll look at the last five months of the year, August through December, encompassing the second wet season and compare this season between two years, 2016 and 2019. 

As always, we'll start by importing the necessary libraries.

<div class="run">
    ▶️ <b> Run the cell below. </b>
</div>

In [ ]:
# Imports
import os
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In this lab, you will use data collected from the flux tower in Kenya to calculate the overall energy balance for this landscape, comparing eddy flux calculations to radiation data. The data are located in the [data](../data) folder of the GitHub repository. The raw eddy covariance data have already been significantly pruned and pre-processed to allow for efficient calculations of turbulent fluxes in half-hourly averages. The data you will need for these calculations are located in a file called [CR3000_SN4709_flux_ed.csv](../data/lab_2/CR3000_SN4709_flux_ed.csv). These data include:

| Column name    | Units        | Symbol | Description |
| :------------- | :---- | :----- | :---------- |
| **P_a**        | $\text{kPa}$ | $P_a$ | Air pressure |
| **e**          | $\text{kPa}$       | $e$   | Vapor pressure |
| **T_a**        | $^{\circ}\text{C}$ | $T_a$ | Air temperature |
| **cov_Ts_Uz**  | $\text{K m s}^{-1}$ | $\overline{w'T'}$ | Eddy covariance term for SHF |
| **cov_h2o_Uz** | $\text{g m}^{-2} \text{ s}^{-1}$ | $\overline{w'c'_{_{\text{H}_2\text{O}}}}$ | Eddy covariance term for LHF |

<br></br>

The other file, [CR5000_SN2446_upper_ed.csv](../data/lab_2/CR5000_SN2446_upper_ed.csv), contains data from a net radiometer that you will use to assess your eddy covariance calculations. This file is organized as follows:

| Column name | Units | Symbol | Description |
| :---------- | :---- | :----- | :---------- |
| **DOY** | | | Day of year |
| **SW_in**  | $\text{W m}^{-2}$ | $R^{\downarrow}_{_{SW}}$ | Incoming shortwave radiation |
| **SW_out** | $\text{W m}^{-2}$ | $R^{\uparrow}_{_{SW}}$ | Outgoing shortwave radiation |
| **LW_in**  | $\text{W m}^{-2}$ | $R^{\downarrow}_{_{LW}}$ | Incoming longwave radiation  |
| **LW_out** | $\text{W m}^{-2}$ | $R^{\uparrow}_{_{LW}}$ | Outgoing longwave radiation  |
| **P_mm**   | $\text{mm}$       | $P$ | Precipitation                |




<div class="run">
    ▶️ <b> Run the cell below. </b>
</div>

In [ ]:
# Data import
flux = pd.read_csv('../data/lab_2/CR3000_SN4709_flux_ed.csv', index_col=0, parse_dates=True)
rad = pd.read_csv('../data/lab_2/CR5000_SN2446_upper_ed.csv', index_col=0, parse_dates=True)

Finally, the following table contains values for several potentially useful constants, which have already been coded in the cell below.

<table>
<thead>
  <tr>
    <th style="text-align:left;"> Constant </th>
    <th style="text-align:center;"> Symbol </th>
    <th style="text-align:left; width:158px"> Value </th>
  </tr>
</thead>
<tbody>
  <tr>
    <td style="text-align:left;"> Stefan-Boltzmann constant </td>
    <td style="text-align:center;"> $\sigma$ </td>
    <td> $5.67 \times 10^{-8} \text{ W m}^{-2} \text{ K}^{-4}$ </td>
  </tr>
  <tr>
    <td style="text-align:left;"> Specific heat capacity of air </td>
    <td style="text-align:center;"> $c_p$ </td>
    <td> $1004.67 \text{ J kg}^{-1} \text{ K}^{-1}$ </td>
  </tr>
  <tr>
    <td style="text-align:left;"> Ideal gas constant </td>
    <td style="text-align:center;"> $R$ </td>
    <td> $8.314 \text{ J mol}^{-1} \text{ K}^{-1}$ </td>
  </tr>
  <tr>
    <td style="text-align:left;"> Latent heat of vaporization </td>
    <td style="text-align:center;"> $\lambda_v$ </td>
    <td> $2440 \text{ J g}^{-1}$ </td>
  </tr>
  <tr>
    <td style="text-align:left;"> Molar mass of water </td>
    <td style="text-align:center;"> $M_{_{\text{H}_2\text{O}}}$ </td>
    <td> $18.01528 \text{ g mol}^{-1}$ </td>
  </tr>
  <tr>
    <td style="text-align:left;"> Molar mass of dry air </td>
    <td style="text-align:center;"> $M_{_{\text{air}}}$ </td>
    <td> $28.9647 \text{ g mol}^{-1}$ </td>
  </tr>
</tbody>
</table>


<div class="run">
    ▶️ <b> Run the cell below. </b>
</div>

In [ ]:
# Constants
sigma = 5.67e-8  # Stefan-Boltzmann constant, W m-2 K-4
c_p = 1004.67    # heat capacity, J kg-1 K-1
R = 8.314        # Ideal gas constant, J mol-1 K-1
lambda_v = 2440  # Latent heat of vaporization, J g-1
M_h2o = 18.01528 # Molar mass of water, g mol-1
M_air = 28.9647  # Molar mass of dry air, g mol-1

# 2. The Radiation Budget ☀️

<hr style="border-top: 0.5px solid gray; margin-top: 10pt; margin-bottom: 12pt"></hr>

The Earth's radiation budget can be broken into *shortwave* and *longwave* components, referring to the wavelength of transmitted light. Each of these components consists of a flux of incoming (or "downwelling") and outgoing (or "upwelling") energy. Thus, we can write the 

\begin{equation}
\tag{2.1}
    R_n \ = \ R^{\downarrow}_{_{SW}} \, - \, R^{\uparrow}_{_{SW}} \, + \, R^{\downarrow}_{_{LW}} \, - \, R^{\uparrow}_{_{LW}}
\end{equation}

where the arrows denote the direction of the radiation. We'll explore each of these components in more detail below.

<img src="./assets/em_spectrum.jpg" alt="spectrum" width="500"/>

## 2.1. Shortwave radiation
The vast majority of energy entering the Earth's atmosphere is **shortwave radiation** from the sun, $R^{\ \downarrow}_{_{SW}}$, and falls in the ultraviolet, visible, and infrared portions of the electromagnetic spectrum. As we saw in Exercise 1.1, the flux density of the sun's energy at the Earth's atmosphere, known as the *solar constant*, is about 1370 W m<sup>-2</sup>. This is the *maximum* value of $R^{\downarrow}_{_{SW}}$, but this quantity can fluctuate based on seasonal and diurnal geometry, attenuation by the atmosphere, and the amount of clouds and aerosols in the atmosphere. Of the solar radiation that *does* pass through the atmosphere, about 30% is reflected by the Earth's surface, leading to **outgoing shortwave radiation, $R^{\uparrow}_{_{SW}}$**. The ratio of reflected to incoming shortwave radiation is known as **albedo**, $\alpha$.

\begin{equation}
\tag{2.2}
    \alpha \ = \ \frac{R^{\uparrow}_{_{SW}}}{R^{\downarrow}_{_{SW}}}
\end{equation}

Because ~30% of incoming solar radiation is reflected on average, the Earth's average global albedo is 0.3. This value varies between 0 and 1, however, based on the surface. Dark objects like asphalt absorb most of the incident radiation causing them to have a very low albedo. In contrast, light objects like freshly fallen snow have an albedo close to 1 as they reflect nearly all incident energy.

Using Equation 2.2, we can write the net shortwave radiation (i.e. the difference between incoming and outgoing shortwave radiation) as

\begin{equation}
\tag{2.3}
    R^{\downarrow}_{_{SW}} \ - \ R^{\uparrow}_{_{SW}} \ = \ (1 \ - \ \alpha) R^{\downarrow}_{_{SW}}
\end{equation}

## 2.2. Longwave radiation

Some incoming solar radiation is absorbed by larger particles in the Earth's atmosphere, causing the atmosphere to warm. The heat generated by this absorption is emitted in the far infrared portion of the elecromagnetic spectrum, some of which radiates to the surface to produce **incoming longwave radiation, $R^{\downarrow}_{_{LW}}$**. The value of $R^{\downarrow}_{_{LW}}$ varies depending on the temperature profile of the atmosphere and the amount of water vapor and other particles present. As the surface of the Earth heats up, thermal energy is re-emitted as **outgoing longwave radiation, $R^{\uparrow}_{_{LW}}$**. This quantity depends on the temperature and **emissivity** of the surface, $\varepsilon$. Both incoming and outgoing longwave radiation can be calculated from the Stefan-Boltzmann law, which describes the energy emitted from an object in terms of its temperature:

\begin{equation}
\tag{2.4}
    j^* \ = \ \varepsilon \sigma T^{4}
\end{equation}

where $\sigma$ is the **Stefan-Boltzman constant** with a value of $5.67 \times 10^{-8} \text{ W m}^{-2} \text{ K}^{-4}$. 

Thus, we can rewrite Equation 2.1 to express net radiation in terms of these relationships in Equations 2.3 and 2.4:

\begin{equation}
\tag{2.5}
    R^{}_{n} \ = \ (1-\alpha) \ R^{\downarrow}_{_{SW}} \ + \ \varepsilon_a \sigma {T^{}_{a}}^{4} \ - \ \varepsilon_s \sigma {T^{}_{s}}^{4}
\end{equation}

where $\varepsilon_a$ and $\varepsilon_s$ are the emissivities and $T_a$ and $T_s$ are the temperatures (in Kelvin) of the air and the surface, respectively.

The sum of the incoming shortwave radiation (corrected for albedo) and incoming longwave radiation represents the **total energy available to the system, $Q_{av}$**. We can rewrite Equation 2.5 in terms of available energy:

\begin{equation}
\tag{2.6}
    Q_{av} \ = \ (1-\alpha) \ R^{\downarrow}_{_{SW}} \ + \ \varepsilon_a \sigma {T^{}_{a}}^{4} \ = \ R^{\downarrow}_{_{SW}} \, - \, R^{\uparrow}_{_{SW}} \, + \, R^{\downarrow}_{_{LW}}  \ = \varepsilon_s \sigma {T^{}_{s}}^{4}
\end{equation}


<hr style="border-top: 0.5px solid gray; margin-top: 6pt; margin-bottom: 12pt"></hr>

## Problems

### 2.1. Determine the albedo of the landscape.

<div class="practice">
    📚  <b> Question 2.1. </b> 
    Calculate daily albedo values using the average daily $R^{\downarrow}_{_{SW}}$ and $R^{\uparrow}_{_{SW}}$ values. What is the mean daily albedo of the landscape over the entire dataset?
</div>

### 2.2. Estimate the surface temperature of the earth from $Q_{av}$

<div class="practice">
    📚  <b> Question 2.2. </b> 
    Calculate $Q_{av}$ from $R^{\downarrow}_{_{SW}}$, $R^{\uparrow}_{_{SW}}$, and $R^{\downarrow}_{_{LW}}$. Using the mean value of $Q_{av}$ over the entire dataset, use Equation 2.6 to calculate the apparent surface temperature of the earth, $T_s$, in both $^{\circ}$C and $^{\circ}$F assuming all of the available energy is used to heat the surface. Assume the surface has the same emissivity as the atmosphere, $\varepsilon_s \ = \ 0.95$.
</div>

### 2.3. Net radiation

Now, calculate the actual net radiation from the dataset.

<div class="practice">
    📚  <b> Question 2.3. </b> 
    Use Equation 2.1 and the actual values of outgoing longwave radiation to calculate net radiation. Plot net radiation from August to December for 2016 and 2019 on the same plot (Hint: use <code>rad.index.dayofyear</code> on the x-axis to display the time series on top of one another). Label your plot appropriately.
</div>

### 2.4. Diurnal radiation

Next, let's look at the diurnal variation in radiation values. Pick a single day of your choosing and plot all four components of radiation, as well as net radiation. You can use a boolean mask on your `DataFrame` to select a single date. For instance, to select the 11th of October 2019:

```python
rad[rad.index.date == datetime.date(2019,10,11)]
```

<div class="practice">
    📚  <b> Question 2.4. </b> 
    Plot net radiation, as well as its four components, over the course of your chosen day. Which component is the largest? When is $R_n$ at its highest during the day? Its lowest? As always, be sure to label your plot appropriately.
</div>

# 3. Landscape energy budget 🏞️

<hr style="border-top: 0.5px solid gray; margin-top: 10pt; margin-bottom: 12pt"></hr>

As we saw above, the amount of energy available to the Earth's surface from radiation far exceeds the outgoing longwave radiation emitted by the surface, so there must be other processes cooling off the surface.  


Energy flows follow a gradient a high to low concentration. We can describe the flow of energy, $F_Q$, in some direction, $x$, in terms of this gradient ($\frac{dQ}{dx}$) and the **conductivity $(k_c)$** of the surface.

\begin{equation}
\tag{3.1}
    F_Q \ = \ - k_c \frac{dQ}{dx}
\end{equation}

This equation is known as **Fick's Law**, and the negative sign reflects the convention that energy flows from high to low. Thus, when the gradient is increasing (positive), flow is directed away (negative). Conductance refers to the ability of the substance or surface to transport energy.

At the surface of the Earth, there are three important transport-driven processes – **conduction**, **convection**, and **evaporation** – each of which is associated with an energy flux.


## 3.1. Ground Heat Flux

The transfer of energy between two materials in contact with one another is known as **conduction**. Conduction is driven by temperature gradients between the two surfaces. In the Earth system context, thermal conduction of heat into the soil is quantified by the **ground heat flux**, $G$. In most landscapes, very little energy is stored in the ground, so $G$ is often ignored in the landscape energy balance.

## 3.2. Sensible Heat Flux 🌡️

**Sensible heat flux**, $H$, refers to the loss of energy from the surface to the atmosphere via **convection**. Like conduction, convection is driven by differences in temperature, but this time between the *surface* and the *air*. That is,

\begin{equation}
\tag{3.2}
    H \ \propto \ T_s - T_a
\end{equation}

This proportionality between sensible heat flux and the temperature difference is determined by the properties of the surface and the turbulent flow across the surface:

\begin{equation}
\tag{3.2}
    H \ = \ k_H \rho_a c_p (T_s - T_a)
\end{equation}

where $k_H$ is the atmospheric conductivity, $\rho_a$ is the density of air, and $c_p$ is the specific heat capacity of air (at constant pressure). Heat capacity refers to the amount of energy required to change the temperature of a substance. Though $c_p$ varies slightly with air temperature, to simplify our calculations we will assume a constant value of $1004.67 \text{ J kg}^{-1} \text{ K}^{-1}$ for $c_p$.

The atmospheric conductivity depends on the roughness of the surface and the wind speed. While we cannot measure this quantity directly, we can use **eddy covariance** to calculate $H$. In the air, heat is transported by the turbulent flow of air in multiple layers. By averaging the product of the fluctuations of temperature and vertical wind, we can determine the amount of energy transported to or from the surface via the air – i.e. the sensible heat flux. Thus, our equation for $H$ becomes

\begin{equation}
\tag{3.3}
    H \ = \ \overline{\rho_{a}} \ c_p \ \overline{w'T'}
\end{equation}

The overbars in this equation represent averages taken over a period of time when the mean vertical flow is negligible (conventionally 30 minutes). This method relies on sensors to measure the fluctuations of temperature, wind, and gas concentrations at very high frequencies in order to adequately sample the eddies that are responsible for transport. The two sensors - an anemometer and a gas analyzer – must be a close together as possible to effectively sample the same pocket of air at the same time.

<img src="./assets/towersensors.jpeg" alt="sensors" width="600"/>

The gas analyzer provides a measurement of the concentration of water vapor in the air, which allows for calculation of air density over time. The average air density over any period of time can be expressed as the sum of the average density of dry air, $\overline{\rho_d}$, and the average vapor density, $\overline{\rho_v}$:

\begin{equation}
\tag{3.4}
    \overline{\rho_{a}} \ = \ \overline{\rho_{v}} + \overline{\rho_d}
\end{equation}

Because we know the concentration of water vapor in the air, we can determine the **atmospheric vapor pressure** (or partial pressure of water vapor in the air), $\overline{e_a}$. Using the ideal gas law, which relates the amount, pressure, and temperature of a gas, we can calculate vapor density:

\begin{equation}
\tag{3.5}
    \overline{\rho_v} \ = \ \frac{M_{_{\text{H}_2\text{O}}} \cdot \overline{e_a}}{R \ \overline{T_a}}
\end{equation}

where $M_{_{\text{H}_2\text{O}}}$ is the molar mass of water, $R$ is the ideal gas constant, and $\overline{T_a}$ is the average air temperature.

Similarly, we can compute the dry air density over the same time period using the partial pressure of dry air, which is simply the difference between total air pressure, $\overline{P_a}$, and the vapor pressure:

\begin{equation}
\tag{3.6}
    \overline{\rho_d} \ = \ \frac{M_{_{\text{air}}} \left( \overline{P_a} \ - \ \overline{e_a}\right)}{R \ \overline{T_a}}
\end{equation}

where $M_{_{\text{air}}}$ is the molar mass of dry air.

## 3.3. Latent Heat Flux 💧

**Latent heat flux**, $\lambda E$, is the energy flux associated with phase changes (**evaporation** and **condensation**) between water at the Earth's surface and water vapor in the air. $\lambda E$ is driven by gradients of *vapor pressure* between the surface ($e_s$) and the air ($e_a$):

\begin{equation}
\tag{3.7}
    \lambda E \ \propto \ e_s - \ e_a
\end{equation}

This relationship also depends on the density and total pressure of the air, the atmospheric conductivity to vapor transport, $k_E$, and the latent heat of vaporization of water, $\lambda_v$ (the energy required to convert one mole of H<sub>2</sub>O from liquid to gas at 100$^{\circ}$C):


\begin{equation}
\tag{3.8}
    \lambda E \ = \ k_E \lambda_v \rho_a \frac{M_{_{\text{air}}}}{M_{_{\text{H}_2\text{O}}}} \frac{e_s - e_a}{P_a}
\end{equation}

Like the sensible heat flux, it is difficult to measure $\lambda E$ directly, but we can calculate it using eddy covariance:

\begin{equation}
\tag{3.9}
    \lambda E \ = \ \overline{\rho_a} \lambda_v \overline{w' \chi_{v}'}
\end{equation}

where $\chi_v$ is the mole fraction of water vapor in the air.

While Equation 3.9 provides a rough approximation of latent heat flux, in practice, a number of corrections must be made to this equation to account for the separation of the sensors and the varying density of air due to both sensible and latent heat fluxes. These corrections have been implemented in a function (`lhf.py`) for ease of our computations. Thus, we can import the function and calculate $\lambda E$ as follows:

```python 
from lhf import lhf_wpl

lhf_wpl(df)
```

For now, we'll just import the function.

<div class="run">
    ▶️ <b> Run the cell below. </b>
</div>

In [ ]:
from lhf import lhf_wpl

<hr style="border-top: 0.5px solid gray; margin-top: 6pt; margin-bottom: 12pt"></hr>

## Problems

### 3.1. Density of air

Before calculating $H$, it's useful to calculate the vapor density and densities of dry and wet air.

<div class="practice">
    📚  <b> Question 3.1. </b> 
    Use equations 3.4, 3.5, and 3.6 to calculate $\rho_v$, $\rho_d$, and $\rho_a$. Hint: be sure to convert temperature to K and check your units!. To make your life easier later on, add $T$ in K and each of the densities to the original <span class="code">DataFrame</span> with the following names: <code>'T_K'</code>, <code>'rho_v'</code>, <code>'rho_d'</code>, and <code>'rho_a'</code>.
</div>

### 3.2. Calculate $H$

Now that you've determined $\rho_a$, you can use this value to calculate sensible heat flux.

<div class="practice">
    📚  <b> Question 3.2. </b> 
    Use Equation 3.3 to determine sensible heat flux, $H$.
</div>

### 3.3. Latent heat flux

<div class="practice">
    📚  <b> Question 3.3. </b> 
    Use the <code>lhf_wpl</code> function to determine latent heat flux, $\lambda E$.
</div>

### 3.4 Comparing turbulent fluxes

Now that you've calculated both sensible and latent heat, compare how the two vary. Consider the conditions under which $H$ and $\lambda E$ are positive or negative. What physical processes drive these fluctuations?

<div class="practice">
    📚  <b> Question 3.4. </b> 
    Plot sensible and latent heat for the last week of each year (beginning on December 25th). Be sure your plot(s) is/are appropriately scaled and labeled. How do they compare? When does $H$ exceed $\lambda E$? When does $\lambda E$ exceed $H$? Why do are the fluxes so different for the same week of 2016 and 2019 (Hint: there is one column of data you've been provided that you have not yet used.)?
</div>

# 4. Putting it all together

<hr style="border-top: 0.5px solid gray; margin-top: 10pt; margin-bottom: 12pt"></hr>

Now that you have all the terms in Equation 1.2 (with the exception of $G$, which we will assume is negligible), you can calculate the overall energy balance of the landscape:

\begin{equation}
\tag{4.1}
    \frac{dQ}{dt} \ = \ R_n \ - \ H \ - \ \lambda E
\end{equation}


In order to compare the radiation data to the flux data, you must first ensure that the two datasets have the same temporal resolution. You can do this in <span class="code">pandas</span> using `df.resample()`, which resamples the dataset over a defined interval. For our dataset, this looks like:

```python
rad30 = rad.resample(pd.tseries.offsets.Minute(30)).mean()
```

<div class="example">
    ✏️ <b> Try it. </b> 
    Copy and paste the code above to generate a new <span class="codeb">DataFrame</span> with half-hourly averages of the radiation data. (Note that this will also resample the precipitation data, which is not useful. You can choose to either ignore this column or remove it from the new <span class="codeb">DataFrame</span>.
</div>

### 4.1. Residual energy

Now that all the data are at the same frequency, calculate $\frac{dQ}{dt}$ using Equation 4.1.

<div class="practice">
    📚  <b> Question 4.1. </b> 
    Calculate the residual energy flux, $\frac{dQ}{dt}$, using Equation 4.1.
</div>

### 4.2. Rainfall

<div class="practice">
    📚  <b> Question 4.2. </b> 
    Calculate daily rainfall amounts and use a bar graph to plot daily rainfall totals on two separate subplots. Which year was wetter? 
</div>

### 4.3. Diurnal radiation, sensible heat flux, and latent heat flux

Examine the figure you generated in 4.2 (and any of the raw data you choose – temperature, pressure, etc.). Pick an interesting day and compare the diurnal variation in net radiation, sensible and latent heat flux, and the residual energy.

<div class="practice">
    📚  <b> Question 4.3. </b> 
    Plot $H$, $\lambda E$, $R_n$, and the net total energy flux (i.e. the quantity you calculated in Question 4.1) across a single day in 2016 and 2019. In which year is the system closer to steady state? Discuss the differences between the two and speculate what is driving these variations.
</div>

### 4.4. Average residual energy

Calculate the average residual energy, $\frac{dQ}{dt}$, for each year.

<div class="practice">
    📚  <b> Question 4.4. </b> 
    Calculate the average residual energy, $\frac{dQ}{dt}$, for each year.
</div>

### 4.5. Energy balance closure.

In addition to the sensors on the flux tower, the field site in Kenya is also equipped with soil heat flux plates that measure (you guessed it) the soil (or ground) heat flux, $G$. Import the soil heat flux data in the cell below (the file has a single column of values of $G$ in W m<sup>-2</sup>).

<div class="run">
    ▶️ <b> Run the cell below. </b>
</div>

In [ ]:
shf = pd.read_csv('../data/lab_2/CR3000_SN9945_Table1_ed.csv',header=0,names=['G'],index_col=0,parse_dates=True)

<div class="practice">
    📚  <b> Question 4.5. </b> 
    Re-calculate the average annual residual energy (i.e. repeat Questions 4.1 and 4.4) using the soil heat flux data and Equation 1.1. Was our previous assumption of negligible soil heat flux valid? If so, suggest a possible explanation for the discrepancy between $R_n$ and $H + \lambda E$. Otherwise, state why you think including $G$ made a difference.
</div>

<hr style="border-top: 1px solid gray; margin-top: 24px; margin-bottom: 1px"></hr>

In [1]:
# IGNORE THIS CELL
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/exercises.css", "r").read()
    return HTML(styles)
css_styling()